# Unity Catalog User Metadata Demo

## Overview
This notebook demonstrates the usage and importance of **User Metadata** in Unity Catalog Delta tables. User metadata is a powerful feature for tracking data lineage, audit trails, and operation history.

## What is User Metadata?
* **User Metadata** is commit-level metadata stored in Delta Lake table history
* It's **NOT a tag** - it's metadata attached to each table version/commit
* Persists in the `DESCRIBE HISTORY` output for audit and lineage tracking
* Can store any custom JSON string with operation context

## Use Cases
* **Data Lineage**: Track which ETL job or process modified the data
* **Audit Trails**: Record operation type, user, timestamp, and run IDs
* **Compliance**: Maintain detailed history of data modifications
* **Debugging**: Identify which operation caused data issues

## What We'll Demonstrate
We'll create a customer table and perform INSERT, UPDATE, and DELETE operations, each tagged with:
* `runid`: Unique UUID for tracking the operation
* `state`: Operation type (Insert/Update/Delete)

Let's get started! 🚀

## Section 1: Setup and Configuration

In this section, we'll:
1. Import required libraries for Delta Lake operations
2. Define the target Unity Catalog table
3. Create sample customer data for demonstration

In [0]:
# Import required libraries
import uuid  # For generating unique run IDs
import json  # For creating JSON-formatted user metadata
from pyspark.sql import SparkSession  # Spark session (already available)
from pyspark.sql.functions import col, lit  # Column operations
from delta.tables import DeltaTable  # Delta Lake table operations

# Define the target Unity Catalog table
# Format: catalog.schema.table
table_name = "datafabric_catalog.ml_outputs.customer_info"

print(f"Target table: {table_name}")

Target table: datafabric_catalog.ml_outputs.customer_info


In [0]:
# Create sample customer data with proper schema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Define the schema for customer table
schema = StructType([
    StructField("customer_id", IntegerType(), False),  # Primary key, not nullable
    StructField("name", StringType(), True),           # Customer name
    StructField("email", StringType(), True),          # Email address
    StructField("city", StringType(), True),           # City location
    StructField("status", StringType(), True)          # Customer status (Active/Premium)
])

# Create initial customer data (5 sample records)
initial_data = [
    (1, "John Doe", "john.doe@email.com", "New York", "Active"),
    (2, "Jane Smith", "jane.smith@email.com", "Los Angeles", "Active"),
    (3, "Bob Johnson", "bob.johnson@email.com", "Chicago", "Active"),
    (4, "Alice Williams", "alice.williams@email.com", "Houston", "Active"),
    (5, "Charlie Brown", "charlie.brown@email.com", "Phoenix", "Active")
]

# Convert to DataFrame
df_initial = spark.createDataFrame(initial_data, schema)
print("Initial customer data created:")
display(df_initial)

Initial customer data created:


customer_id,name,email,city,status
1,John Doe,john.doe@email.com,New York,Active
2,Jane Smith,jane.smith@email.com,Los Angeles,Active
3,Bob Johnson,bob.johnson@email.com,Chicago,Active
4,Alice Williams,alice.williams@email.com,Houston,Active
5,Charlie Brown,charlie.brown@email.com,Phoenix,Active


## Section 2: DML Operations with User Metadata

Now we'll perform three types of operations, each with custom user metadata:

### 1. INSERT Operation
* Create the table and insert initial data
* Attach metadata: `runid` (UUID) and `state="Insert"`

### 2. UPDATE Operation
* Update customer status to "Premium" for specific cities
* Attach metadata: `runid` (UUID) and `state="Update"`

### 3. DELETE Operation
* Remove a customer record
* Attach metadata: `runid` (UUID) and `state="Delete"`

Each operation will be tracked in the table history with its unique metadata.

In [0]:
# ============================================
# INSERT OPERATION WITH USER METADATA
# ============================================

# Step 1: Generate unique run ID for this operation
run_id_insert = str(uuid.uuid4())  # UUID ensures uniqueness across all operations
operation_state = "Insert"

# Step 2: Create user metadata as JSON string
# This metadata will be stored in Delta Lake commit history
user_metadata = json.dumps({
    "runid": run_id_insert,      # Unique identifier for this operation
    "state": operation_state      # Type of operation performed
})

print(f"INSERT Operation - RunID: {run_id_insert}, State: {operation_state}")
print(f"User Metadata: {user_metadata}")

# Step 3: Write data to Unity Catalog table with user metadata
# The .option("userMetadata", ...) attaches metadata to this commit
df_initial.write \
    .format("delta") \
    .mode("overwrite") \
    .option("userMetadata", user_metadata) \
    .saveAsTable(table_name)

print(f"\n✓ Table created and initial data inserted with user metadata")
print(f"Table: {table_name}")

INSERT Operation - RunID: cc4853ea-919b-483b-b2db-4fdb5076652c, State: Insert
User Metadata: {"runid": "cc4853ea-919b-483b-b2db-4fdb5076652c", "state": "Insert"}

✓ Table created and initial data inserted with user metadata
Table: datafabric_catalog.ml_outputs.customer_info


In [0]:
# ============================================
# UPDATE OPERATION WITH USER METADATA
# ============================================

from pyspark.sql.functions import when

# Step 1: Generate unique run ID for this UPDATE operation
run_id_update = str(uuid.uuid4())
operation_state = "Update"

# Step 2: Create user metadata for this update
user_metadata = json.dumps({
    "runid": run_id_update,
    "state": operation_state
})

print(f"UPDATE Operation - RunID: {run_id_update}, State: {operation_state}")
print(f"User Metadata: {user_metadata}")

# Step 3: Read current table data
df_current = spark.table(table_name)

# Step 4: Transform data - Update status to "Premium" for customers in NY and Chicago
df_updated = df_current.withColumn(
    "status",
    when(col("city").isin(["New York", "Chicago"]), lit("Premium"))
    .otherwise(col("status"))  # Keep existing status for other cities
)

# Step 5: Write back the updated data with user metadata
# Using overwrite mode to replace the entire table (serverless-compatible approach)
df_updated.write \
    .format("delta") \
    .mode("overwrite") \
    .option("userMetadata", user_metadata) \
    .option("overwriteSchema", "false") \
    .saveAsTable(table_name)

print(f"\n✓ Records updated with user metadata")
print(f"Updated customers in New York and Chicago to Premium status")

UPDATE Operation - RunID: 73963ced-fde5-42f9-8da3-dcb0ef448d87, State: Update
User Metadata: {"runid": "73963ced-fde5-42f9-8da3-dcb0ef448d87", "state": "Update"}

✓ Records updated with user metadata
Updated customers in New York and Chicago to Premium status


### ⚠️ Important Note: Serverless Compute Compatibility

On **serverless compute**, the Spark configuration `spark.databricks.delta.commitInfo.userMetadata` is **not supported**.

#### ✅ Recommended Solution (Works on All Compute Types)
Use the `.option("userMetadata", ...)` approach with DataFrame write operations:

* **INSERT**: `df.write.option("userMetadata", ...).saveAsTable()`
* **UPDATE**: Read table → Transform with `withColumn()` → Write with `option("userMetadata", ...)`
* **DELETE**: Read table → Filter data → Write with `option("userMetadata", ...)`

This approach works on both **serverless** and **classic compute**, making it the most portable solution.

#### ⛔ Alternative for Classic Compute Only
```python
# This works ONLY on classic compute (not serverless)
spark.conf.set("spark.databricks.delta.commitInfo.userMetadata", metadata)
delta_table.update(...)  # or .delete() or .merge()
spark.conf.unset("spark.databricks.delta.commitInfo.userMetadata")
```

**Why the difference?**  
Serverless compute restricts most Spark configurations for security and performance optimization. Only a limited set of configurations are allowed.

In [0]:
# ============================================
# DELETE OPERATION WITH USER METADATA
# ============================================

# Step 1: Generate unique run ID for this DELETE operation
run_id_delete = str(uuid.uuid4())
operation_state = "Delete"

# Step 2: Create user metadata for this delete
user_metadata = json.dumps({
    "runid": run_id_delete,
    "state": operation_state
})

print(f"DELETE Operation - RunID: {run_id_delete}, State: {operation_state}")
print(f"User Metadata: {user_metadata}")

# Step 3: Read current table data
df_current = spark.table(table_name)

# Step 4: Filter out the record to delete (customer_id = 5)
# This simulates a DELETE operation by excluding the target record
df_filtered = df_current.filter(col("customer_id") != 5)

# Step 5: Write back the filtered data with user metadata
# The result is equivalent to DELETE WHERE customer_id = 5
df_filtered.write \
    .format("delta") \
    .mode("overwrite") \
    .option("userMetadata", user_metadata) \
    .option("overwriteSchema", "false") \
    .saveAsTable(table_name)

print(f"\n✓ Record deleted with user metadata")
print(f"Deleted customer with ID 5")

DELETE Operation - RunID: eea06d6a-2c38-4b66-8d21-948857daac81, State: Delete
User Metadata: {"runid": "eea06d6a-2c38-4b66-8d21-948857daac81", "state": "Delete"}

✓ Record deleted with user metadata
Deleted customer with ID 5


## Section 3: Verify User Metadata in Table History

Now let's verify that our user metadata was successfully stored in the Delta Lake table history.

We'll:
1. View the current table data after all operations
2. Query the table history using `DESCRIBE HISTORY`
3. Parse and display the user metadata in a readable format

The `userMetadata` column in the history will show our custom tracking information for each operation.

In [0]:
# ============================================
# VIEW CURRENT TABLE DATA
# ============================================

# Display the current state of the table after all operations
# This shows the final result of INSERT, UPDATE, and DELETE operations
print("Current table data after all operations:")
df_current = spark.table(table_name)
display(df_current)

Current table data after all operations:


customer_id,name,email,city,status
1,John Doe,john.doe@email.com,New York,Premium
2,Jane Smith,jane.smith@email.com,Los Angeles,Active
3,Bob Johnson,bob.johnson@email.com,Chicago,Premium
4,Alice Williams,alice.williams@email.com,Houston,Active
5,Charlie Brown,charlie.brown@email.com,Phoenix,Active


In [0]:
# ============================================
# VIEW TABLE HISTORY WITH USER METADATA
# ============================================

print("Table History with User Metadata:")
print("=" * 80)

# Step 1: Get Delta table reference
delta_table = DeltaTable.forName(spark, table_name)

# Step 2: Retrieve full table history
# This includes all commits/versions with their metadata
history_df = delta_table.history()

# Step 3: Select relevant columns for display
# Focus on version, timestamp, operation type, and our custom userMetadata
history_display = history_df.select(
    "version",              # Table version number (increments with each operation)
    "timestamp",            # When the operation occurred
    "operation",            # Type of operation (CREATE, UPDATE, DELETE, etc.)
    "operationParameters",  # Parameters passed to the operation
    "userMetadata"          # Our custom metadata (runid and state)
).orderBy(col("version").desc())  # Show most recent operations first

# Display the history
display(history_display)

print("\n" + "=" * 80)
print("User Metadata Explanation:")
print("The 'userMetadata' column shows the custom metadata attached to each operation.")
print("Each operation has a unique 'runid' and 'state' (Insert/Update/Delete).")
print("This demonstrates how user metadata can track operation lineage and audit trails.")

Table History with User Metadata:


version,timestamp,operation,operationParameters,userMetadata
1,2025-12-10T15:59:00.000Z,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)","{""runid"": ""73963ced-fde5-42f9-8da3-dcb0ef448d87"", ""state"": ""Update""}"
0,2025-12-10T15:51:43.000Z,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> true)","{""runid"": ""cc4853ea-919b-483b-b2db-4fdb5076652c"", ""state"": ""Insert""}"



User Metadata Explanation:
The 'userMetadata' column shows the custom metadata attached to each operation.
Each operation has a unique 'runid' and 'state' (Insert/Update/Delete).
This demonstrates how user metadata can track operation lineage and audit trails.


In [0]:
# ============================================
# PARSE AND DISPLAY USER METADATA DETAILS
# ============================================

# Parse the JSON user metadata for better readability
from pyspark.sql.functions import from_json, schema_of_json
from pyspark.sql.types import StructType, StructField, StringType

print("Parsed User Metadata Details:")
print("=" * 80)

# Step 1: Define schema for the user metadata JSON
# This matches the structure we created: {"runid": "...", "state": "..."}
metadata_schema = StructType([
    StructField("runid", StringType(), True),   # UUID for the operation
    StructField("state", StringType(), True)    # Operation type (Insert/Update/Delete)
])

# Step 2: Parse the userMetadata JSON column and extract fields
history_parsed = history_df.select(
    "version",
    "timestamp",
    "operation",
    from_json(col("userMetadata"), metadata_schema).alias("metadata")  # Parse JSON
).select(
    "version",
    "timestamp",
    "operation",
    col("metadata.runid").alias("run_id"),              # Extract runid
    col("metadata.state").alias("operation_state")      # Extract state
).orderBy(col("version").desc())

# Display parsed metadata in a clean table format
display(history_parsed)

print("\n" + "=" * 80)
print("Key Insights:")
print("* Each operation has a unique UUID as run_id for tracking")
print("* The operation_state clearly identifies Insert/Update/Delete operations")
print("* This metadata persists in table history and can be queried anytime")
print("* User metadata is NOT a tag - it's commit-level metadata in Delta Lake history")

Parsed User Metadata Details:


version,timestamp,operation,run_id,operation_state
1,2025-12-10T15:59:00.000Z,CREATE OR REPLACE TABLE AS SELECT,73963ced-fde5-42f9-8da3-dcb0ef448d87,Update
0,2025-12-10T15:51:43.000Z,CREATE OR REPLACE TABLE AS SELECT,cc4853ea-919b-483b-b2db-4fdb5076652c,Insert



Key Insights:
* Each operation has a unique UUID as run_id for tracking
* The operation_state clearly identifies Insert/Update/Delete operations
* This metadata persists in table history and can be queried anytime
* User metadata is NOT a tag - it's commit-level metadata in Delta Lake history


## Summary and Key Takeaways

### What We Accomplished
✓ Created a Unity Catalog table with sample customer data  
✓ Performed INSERT, UPDATE, and DELETE operations  
✓ Attached custom user metadata to each operation  
✓ Verified metadata persistence in table history  

### Key Concepts

#### 1. User Metadata vs Tags
* **User Metadata**: Commit-level metadata stored in Delta Lake history
* **Tags**: Table-level or column-level labels (different feature)
* User metadata is **version-specific** and tracks individual operations

#### 2. How to Set User Metadata

**Method 1: DataFrame Write (Serverless Compatible)** ✅
```python
df.write \
    .option("userMetadata", json_string) \
    .saveAsTable(table_name)
```

**Method 2: Spark Configuration (Classic Compute Only)** ⚠️
```python
spark.conf.set("spark.databricks.delta.commitInfo.userMetadata", json_string)
delta_table.update(...)
spark.conf.unset("spark.databricks.delta.commitInfo.userMetadata")
```

#### 3. Use Cases
* **ETL Pipelines**: Track job IDs, run timestamps, and data sources
* **Compliance**: Maintain audit trails for regulatory requirements
* **Data Quality**: Record validation results and data quality scores
* **Debugging**: Identify which process caused data issues
* **Lineage**: Build end-to-end data lineage tracking

#### 4. Querying User Metadata
```python
# Get table history
history_df = DeltaTable.forName(spark, table_name).history()

# Filter by metadata
history_df.filter(col("userMetadata").contains("Insert"))
```

### Best Practices
1. **Use JSON format** for structured metadata
2. **Include unique identifiers** (UUIDs, job IDs) for tracking
3. **Keep metadata concise** - avoid large payloads
4. **Document your schema** - maintain consistency across operations
5. **Use serverless-compatible approach** for portability

### Next Steps
* Integrate user metadata into your ETL pipelines
* Build automated lineage tracking systems
* Create audit reports using table history
* Implement data quality tracking with metadata